In [22]:
import numpy as np
import ipywidgets as widgets
import plotly.graph_objects as go
from scipy.stats import entropy
from scipy.special import rel_entr
from IPython.display import display, HTML

# --- Core Logic ---
def calculate_entropy(probs):
    """Calculates Shannon Entropy in bits (base 2)."""
    p = np.array(probs)
    # Avoid division by zero if all weights are 0 (edge case)
    if np.sum(p) == 0:
        return 0.0, p
    p = p / np.sum(p)
    return entropy(p, base=2), p

def calculate_kl(p_probs, q_probs):
    """Calculates KL Divergence."""
    return np.sum(rel_entr(p_probs, q_probs))

# --- Interactive UI Class ---
class InfoTheoryDemo:
    def __init__(self):
        self.out_entropy = widgets.Output()
        self.out_kl = widgets.Output()

        # --- Entropy Controls ---
        # continuous_update=False ensures the graph only updates when you release the mouse
        self.w1 = widgets.FloatSlider(value=2, min=0.1, max=10, description='Weight A:', continuous_update=False)
        self.w2 = widgets.FloatSlider(value=5, min=0.1, max=10, description='Weight B:', continuous_update=False)
        self.w3 = widgets.FloatSlider(value=3, min=0.1, max=10, description='Weight C:', continuous_update=False)
        
        # --- KL Divergence Controls ---
        self.p_slider = widgets.FloatSlider(value=0.4, min=0.01, max=0.99, step=0.01, description='P(Event 1):', continuous_update=False)
        self.q_slider = widgets.FloatSlider(value=0.5, min=0.01, max=0.99, step=0.01, description='Q(Event 1):', continuous_update=False)
        
        # --- Bind Events ---
        self.w1.observe(self.update_entropy, names='value')
        self.w2.observe(self.update_entropy, names='value')
        self.w3.observe(self.update_entropy, names='value')
        
        self.p_slider.observe(self.update_kl, names='value')
        self.q_slider.observe(self.update_kl, names='value')

        # Run initial updates
        self.update_entropy(None)
        self.update_kl(None)

    def update_entropy(self, change):
        with self.out_entropy:
            self.out_entropy.clear_output(wait=True)
            
            # 1. Calculate new values
            raw = [self.w1.value, self.w2.value, self.w3.value]
            ent_val, p_norm = calculate_entropy(raw)
            
            # 2. Re-draw Figure
            fig = go.Figure(data=[go.Bar(
                x=['A', 'B', 'C'],
                y=p_norm,
                text=np.round(p_norm, 2),
                textposition='auto',
                marker_color=['#FF9999', '#66B2FF', '#99FF99']
            )])
            fig.update_layout(
                title=f"Probability Distribution<br>Entropy: {ent_val:.4f} bits",
                yaxis_title="Probability",
                yaxis_range=[0, 1],
                height=300,
                margin=dict(l=20, r=20, t=60, b=20)
            )
            fig.show()

    def update_kl(self, change):
        with self.out_kl:
            self.out_kl.clear_output(wait=True)
            
            # 1. Calculate new values
            p_val = self.p_slider.value
            q_val = self.q_slider.value
            
            P = [p_val, 1 - p_val]
            Q = [q_val, 1 - q_val]
            
            kl_val = calculate_kl(P, Q)
            
            # 2. Re-draw Figure
            fig = go.Figure(data=[
                go.Bar(name='P (Reference)', x=['Event 1', 'Event 2'], y=P, marker_color='#66B2FF'),
                go.Bar(name='Q (Approximation)', x=['Event 1', 'Event 2'], y=Q, marker_color='#FF9999')
            ])
            fig.update_layout(
                title=f"Comparing Distributions P vs Q<br>KL Divergence: {kl_val:.4f} nats",
                barmode='group',
                yaxis_range=[0, 1],
                height=300,
                margin=dict(l=20, r=20, t=60, b=20)
            )
            fig.show()

    def render(self):
        # Layout for Entropy Tab
        ent_ui = widgets.VBox([
            widgets.HTML("<b>Adjust Weights (Distribution is auto-normalized):</b>"),
            self.w1, self.w2, self.w3, 
            self.out_entropy
        ])
        
        # Layout for KL Tab
        kl_ui = widgets.VBox([
            widgets.HTML("<b>Adjust Probability of Event 1 for P and Q:</b>"),
            self.p_slider, self.q_slider, 
            self.out_kl
        ])
        
        # Create Tabs
        tabs = widgets.Tab(children=[ent_ui, kl_ui])
        tabs.set_title(0, 'Entropy Calculator')
        tabs.set_title(1, 'KL Divergence')
        
        display(HTML("<h3>📏 Simple Information Theory Demo</h3>"))
        display(tabs)

# --- Run the App ---
app = InfoTheoryDemo()
app.render()